In [ ]:
# Connect to the MySQL database using .env file
import os
from datetime import datetime

import mysql.connector
import pandas as pd
from dotenv import load_dotenv
from mysql.connector import Error

# Load environment variables from .env file
load_dotenv()

True

In [6]:
try:
    connection = mysql.connector.connect(
        host=os.getenv("DB_HOST"),
        database=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
    )

    if connection.is_connected():
        db_info = connection.get_server_info()
        print(f"Connected to MySQL Server version {db_info}")
        cursor = connection.cursor()
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print(f"Connected to database: {record[0]}")

except Error as e:
    print(f"Error connecting to MySQL: {e}")

Connected to MySQL Server version 9.2.0
Connected to database: superstore_dw


In [7]:
# Read the CSV file
df = pd.read_csv("Sample - Superstore.csv", encoding="windows-1252")

# Display the first few rows and column names
print("Column names:", df.columns.tolist())
print("\nFirst 5 rows:")
df.head()

Column names: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']

First 5 rows:


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


## **Dimension Tables ETL**

In [ ]:
def create_level_mappings(df):
    """Create mappings for level keys"""
    # Create sub-category ID mapping
    sub_categories = df["Sub-Category"].drop_duplicates().reset_index(drop=True)
    sub_category_mapping = {
        sub_cat: idx + 1 for idx, sub_cat in enumerate(sub_categories)
    }

    # Create country ID mapping
    countries = df["Country"].drop_duplicates().reset_index(drop=True)
    country_mapping = {country: idx + 1 for idx, country in enumerate(countries)}

    # Create city ID mapping
    city_states = df[["City", "State"]].drop_duplicates().reset_index(drop=True)
    city_mapping = {}
    for idx, (_, row) in enumerate(city_states.iterrows(), 1):
        city_mapping[(row["City"], row["State"])] = idx

    return {
        "sub_category": sub_category_mapping,
        "country": country_mapping,
        "city": city_mapping,
    }


# Function to load data into Calendar dimension table
def load_calendar_dimension(connection, df):
    # Extract unique dates from Order Date and Ship Date
    order_dates = pd.to_datetime(df["Order Date"]).dt.date.unique()
    ship_dates = pd.to_datetime(df["Ship Date"]).dt.date.unique()
    all_dates = sorted(set(order_dates) | set(ship_dates))

    # Create year level mapping (sequential IDs for each year)
    years = sorted(set([d.year for d in all_dates]))
    year_mapping = {year: idx for idx, year in enumerate(years, 1)}

    # Create calendar dataframe
    calendar_data = []
    for date in all_dates:
        dt = datetime.combine(date, datetime.min.time())
        calendar_data.append(
            {
                "full_date": date,
                "year_id": year_mapping[
                    dt.year
                ],  # Use mapped year_id instead of actual year
                "year_number": dt.year,  # Keep actual year as year_number
                "month_id": dt.month,
                "month_number": dt.month,
                "month_name": dt.strftime("%B"),
                "day_id": dt.day,
                "day_number": dt.day,
            }
        )

    calendar_df = pd.DataFrame(calendar_data)

    # Insert data into Calendar table
    cursor = connection.cursor()
    for _, row in calendar_df.iterrows():
        query = """
        INSERT INTO Calendar (full_date, year_id, year_number, month_id, month_number, month_name, day_id, day_number)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(
            query,
            (
                row["full_date"],
                row["year_id"],
                row["year_number"],
                row["month_id"],
                row["month_number"],
                row["month_name"],
                row["day_id"],
                row["day_number"],
            ),
        )

    connection.commit()
    print(f"Loaded {len(calendar_df)} records into Calendar dimension")

    # Now populate CalendarMonth table with unique year-month combinations
    month_data = calendar_df[
        ["year_id", "year_number", "month_id", "month_number", "month_name"]
    ].drop_duplicates()

    for _, row in month_data.iterrows():
        query = """
        INSERT INTO CalendarMonth (calendar_month_number, calendar_month_name, year_id, year_number)
        VALUES (%s, %s, %s, %s)
        """
        cursor.execute(
            query,
            (
                row["month_number"],
                row["month_name"],
                row["year_id"],  # Using consistent year_id from the mapping
                row["year_number"],
            ),
        )

    connection.commit()
    print(f"Loaded {len(month_data)} records into CalendarMonth dimension")

    # Return the year mapping in case needed elsewhere
    return year_mapping


# Function to load Customer dimension table
def load_customer_dimension(connection, df):
    # Extract unique customer data
    customer_df = df[["Customer ID", "Customer Name", "Segment"]].drop_duplicates()

    # Insert data into Customer table
    cursor = connection.cursor()
    for _, row in customer_df.iterrows():
        query = """
        INSERT INTO Customer (customer_code, customer_name, segment)
        VALUES (%s, %s, %s)
        """
        cursor.execute(
            query, (row["Customer ID"], row["Customer Name"], row["Segment"])
        )

    connection.commit()
    print(f"Loaded {len(customer_df)} records into Customer dimension")


# Function to load Region, State, and Location dimension tables
def load_geography_dimensions(connection, df, level_mappings):
    # Extract unique regions and countries
    region_df = df[["Region", "Country"]].drop_duplicates()

    # Insert data into Region table
    cursor = connection.cursor()
    for _, row in region_df.iterrows():
        country_id = level_mappings["country"][row["Country"]]
        query = """
        INSERT INTO Region (region_name, country_id, country_name)
        VALUES (%s, %s, %s)
        """
        cursor.execute(query, (row["Region"], country_id, row["Country"]))

    connection.commit()
    print(f"Loaded {len(region_df)} records into Region dimension")

    # Extract unique state-region combinations
    state_df = df[["State", "Region", "Country"]].drop_duplicates()

    # Get region IDs
    cursor.execute("SELECT region_id, region_name FROM Region")
    region_mapping = {row[1]: row[0] for row in cursor.fetchall()}

    # Insert data into State table
    for _, row in state_df.iterrows():
        region_id = region_mapping.get(row["Region"])
        country_id = level_mappings["country"][row["Country"]]

        query = """
        INSERT INTO State (state_name, region_id, region_name, country_id, country_name)
        VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(
            query, (row["State"], region_id, row["Region"], country_id, row["Country"])
        )

    connection.commit()
    print(f"Loaded {len(state_df)} records into State dimension")

    # Now load Location table
    location_df = df[["Postal Code", "City", "State", "Country", "Region"]].drop_duplicates()

    # Get state IDs
    cursor.execute("SELECT state_id, state_name FROM State")
    state_mapping = {row[1]: row[0] for row in cursor.fetchall()}

    # Insert data into Location table
    for _, row in location_df.iterrows():
        country_id = level_mappings["country"][row["Country"]]
        state_id = state_mapping.get(row["State"])
        city_id = level_mappings["city"][(row["City"], row["State"])]
        region_id = region_mapping.get(row["Region"])

        query = """
        INSERT INTO Location (location_code, country_id, country_name, state_id, state_name, city_id, city_name, postal_code, region_id, region_name)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(
            query,
            (
                row["Postal Code"],
                country_id,
                row["Country"],
                state_id,
                row["State"],
                city_id,
                row["City"],
                row["Postal Code"],
                region_id,
                row["Region"],
            ),
        )

    connection.commit()
    print(f"Loaded {len(location_df)} records into Location dimension")


# Function to load Category and Product dimension tables
def load_product_dimensions(connection, df, level_mappings):
    # Extract unique categories
    category_df = df["Category"].drop_duplicates().reset_index(drop=True)

    # Insert data into Category table
    cursor = connection.cursor()
    for category in category_df:
        query = """
        INSERT INTO Category (category_name)
        VALUES (%s)
        """
        cursor.execute(query, (category,))

    connection.commit()
    print(f"Loaded {len(category_df)} records into Category dimension")

    # Get category IDs for mapping
    cursor.execute("SELECT category_id, category_name FROM Category")
    category_mapping = {row[1]: row[0] for row in cursor.fetchall()}

    # Extract unique products
    product_df = df[
        ["Product ID", "Product Name", "Category", "Sub-Category"]
    ].drop_duplicates()

    # Insert data into Product table
    for _, row in product_df.iterrows():
        category_id = category_mapping.get(row["Category"])
        sub_category_id = level_mappings["sub_category"][row["Sub-Category"]]

        query = """
        INSERT INTO Product (product_code, product_name, category_id, category_name, sub_category_id, sub_category_name)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(
            query,
            (
                row["Product ID"],
                row["Product Name"],
                category_id,
                row["Category"],
                sub_category_id,
                row["Sub-Category"],
            ),
        )

    connection.commit()
    print(f"Loaded {len(product_df)} records into Product dimension")


# Now execute all of our dimension loading functions
try:
    # Make sure we're connected
    if connection.is_connected():
        print("Loading dimension tables...")

        # Create level key mappings first
        level_mappings = create_level_mappings(df)

        load_calendar_dimension(connection, df)
        load_customer_dimension(connection, df)
        load_geography_dimensions(connection, df, level_mappings)
        load_product_dimensions(connection, df, level_mappings)

        print("All dimension tables loaded successfully!")
except Error as e:
    print(f"Error: {e}")

Loading dimension tables...
Loaded 1434 records into Calendar dimension
Loaded 49 records into CalendarMonth dimension
Loaded 793 records into Customer dimension
Loaded 4 records into Region dimension
Loaded 49 records into State dimension
Loaded 632 records into Location dimension
Loaded 3 records into Category dimension
Loaded 1894 records into Product dimension
All dimension tables loaded successfully!


## **Fact Tables ETL**